In [ ]:
import os
import torch
import sys
sys.path.append('/home/sxk230060/TI2AV/Pyramid-Flow/')
from video_vae import CausalVideoVAELossWrapper
from diffusers.utils import export_to_video
from IPython.display import HTML

In [ ]:
model_path = "/mnt/sda1/saksham/TI2AV/pyramid/causal_video_vae"   # The video-vae checkpoint dir
model_dtype = 'bf16'

device_id = 0
torch.cuda.set_device(device_id)

model = CausalVideoVAELossWrapper(
    model_path,
    model_dtype,
    interpolate=False, 
    add_discriminator=False,
)
model = model.to("cuda")

if model_dtype == "bf16":
    torch_dtype = torch.bfloat16 
elif model_dtype == "fp16":
    torch_dtype = torch.float16
else:
    torch_dtype = torch.float32

def show_video(ori_path, rec_path, width="100%"):
    html = ''
    if ori_path is not None:
        html += f"""<video controls="" name="media" data-fullscreen-container="true" width="{width}">
        <source src="{ori_path}" type="video/mp4">
        </video>
        """
    
    html += f"""<video controls="" name="media" data-fullscreen-container="true" width="{width}">
    <source src="{rec_path}" type="video/mp4">
    </video>
    """
    return HTML(html)

In [ ]:
latent_dir = '/mnt/sda1/saksham/TI2AV/AVSync15/latent'
files = os.listdir(latent_dir)

In [ ]:
i = 20
latent_path = os.path.join(latent_dir, files[i])
latent = torch.load(latent_path).to(torch_dtype).to("cuda")

frame_number = 17   # x*8 + 1
width = 256
height = 256
FPS = 8

with torch.no_grad(), torch.cuda.amp.autocast(enabled=True, dtype=torch.bfloat16):
    # latent = latent.unsqueeze(0)
    rec_frames = model.decode_latent(latent.float(), window_size=2, temporal_chunk=True)

export_to_video(rec_frames, "./rec_video.mp4", fps=FPS)
show_video(None, "./rec_video.mp4", "60%")